# Spatially Assign Work¶
In this example, assignments will be assigned to specific workers based on the city district that it falls in. A layer in ArcGIS Online representing the city districts in San Diego will be used.

* Note: This example requires having Arcpy or Shapely installed in the Python environment.

In [1]:
from datetime import datetime
from arcgis.gis import GIS
from arcgis.geometry import Geometry
from arcgis.mapping import WebMap
from arcgis.apps import workforce
from datetime import datetime


### Connect to organization and Get the Project
Let's connect to ArcGIS Online and get the Project with assignments.

In [2]:
gis = GIS("https://arcgis.com", "workforce_scripts")
item = gis.content.get("c765482bd0b9479b9104368da54df90d")
project = workforce.Project(item)

Enter password: ········


### Get Layer of City Districts
Let's get the layer representing city districts and display it.

In [3]:
grid_layer = gis.content.get("8a79535e0dc04410b5564c0e45428a2c").layers[0]
grid_map = gis.map("Palm Springs, CA", zoomlevel=10)
grid_map.add_layer(grid_layer)
grid_map

MapView(layout=Layout(height='400px', width='100%'), zoom=10.0)

### Add Assignments to the Map

In [4]:
grid_map.add_layer(project.assignments_layer)

### Assign Assignments Based on Which District They Intersect¶
Let's fetch the districts layer and query to get all of the districts. Then, for each unassigned assignment intersect the assignment with all districts to determine which district it falls in. Assignments in the yellow district (3) should be assigned to James. Assignments in the red district (8) should be assigned to Aaron. Finally update all of the assignments using "batch_update".

In [5]:
aaron = project.workers.get(user_id="aaron_nitro")
james = project.workers.get(user_id="james_Nitro")

In [6]:
paint_crosswalk = project.assignment_types.get(name="Paint Crosswalk")
assignments = project.assignments.search(f"status=0 AND assignmentType={paint_crosswalk.code}")


### Verify Assignments are Assigned

In [8]:
for assignment in assignments:
    contains = grid_df["SHAPE"].geom.contains(Geometry(assignment.geometry))
    containers = grid_df[contains]
    if not containers.empty: 
        grid = containers['ID'].iloc[0]
        if grid == 10:
            assignment.worker = james
            assignment.status = "assigned"
            assignment.assigned_date = datetime.utcnow()
        elif district == 6:
            assignment.worker = aaron
            assignment.status = "assigned"
            assignment.assigned_date = datetime.utcnow()

# Update the assignments in the project
assignments = project.assignments.batch_update(assignments)

In [9]:
webmap = gis.map("Palm Springs", zoomlevel=11)
webmap.add_layer(project.assignments_layer)
webmap

MapView(layout=Layout(height='400px', width='100%'), zoom=11.0)

In [10]:
import arcgis

gis = arcgis.gis.GIS("https://arcgis.com", "workforce_scripts", "esri12345")
item = gis.content.get("8a79535e0dc04410b5564c0e45428a2c")

item.layers[0].query(as_df=True)

,ID,OID,SHAPE,Shape__Area,Shape__Length
0,1,1,"{""rings"": [[[-12978141.8238, 3994514.2932], [-...",2.659381e+07,20766.7062
1,2,2,"{""rings"": [[[-12972350.4001, 3994514.2932], [-...",2.659381e+07,20766.7064
2,3,3,"{""rings"": [[[-12966558.9763, 3994514.2932], [-...",2.659381e+07,20766.7062
3,4,4,"{""rings"": [[[-12960767.5526, 3994514.2932], [-...",2.659381e+07,20766.7064
4,5,5,"{""rings"": [[[-12978141.8238, 3999106.2226], [-...",2.659381e+07,20766.7062
5,6,6,"{""rings"": [[[-12972350.4001, 3999106.2226], [-...",2.659381e+07,20766.7064
6,7,7,"{""rings"": [[[-12966558.9763, 3999106.2226], [-...",2.659381e+07,20766.7062
7,8,8,"{""rings"": [[[-12960767.5526, 3999106.2226], [-...",2.659381e+07,20766.7064
8,9,9,"{""rings"": [[[-12978141.8238, 4003698.152], [-1...",2.659381e+07,20766.7060
9,10,10,"{""rings"": [[[-12972350.4001, 4003698.152], [-1...",2.659381e+07,20766.7062


In [11]:
project.assignments.batch_delete(assignnments)

NameError: name 'assignnments' is not defined